# Scraping property listings 

### This notebook is to scrape all the listings for properties for sale on Daft.ie 

In [1]:
from playwright.async_api import async_playwright

In [2]:
playwright = await async_playwright().start()

In [3]:
browser = await playwright.chromium.launch(headless = False)

In [4]:
page = await browser.new_page()

In [5]:
await page.goto("https://www.daft.ie/")

<Response url='https://www.daft.ie/' request=<Request url='https://www.daft.ie/' method='GET'>>

In [6]:
# press accept on that message
try: 
    await page.click("xpath=/html/body/div[1]/div/div/main/div/button[2]", timeout=1000)
except:
    pass

In [7]:
await page.goto("https://www.daft.ie/property-for-sale/dublin")

<Response url='https://www.daft.ie/property-for-sale/dublin' request=<Request url='https://www.daft.ie/property-for-sale/dublin' method='GET'>>

In [8]:
# close another pop up that sometimes comes up 
try:
    await page.click("xpath=/html/body/div[5]/div/div/div/div/button/span", timeout=1000)
except: 
    pass

In [9]:
# close another pop up that sometimes comes up 
try: 
    await page.click("xpath=//html/body/div[5]/div/div/div[1]/button", timeout=1000)
except:
    pass

In [10]:
import pandas as pd


In [11]:
from bs4 import BeautifulSoup

In [12]:
html = await page.content()
doc = BeautifulSoup(html)

In [13]:
# get the div which contains the information for each listing 
# content_card = doc.find_all(".Cardstyled__TitleBlockWrapper-nngi4q-4.eMeJos")

content_card = doc.find_all(attrs={"class": ["Cardstyled__TitleBlockWrapper-nngi4q-4","eMeJos"]})

content_card

[<div class="Cardstyled__TitleBlockWrapper-nngi4q-4 eMeJos"><div data-testid="title-block"><div class="TitleBlock__Price-sc-1avkvav-4 hiFkJc" data-testid="price"><span class="TitleBlock__StyledSpan-sc-1avkvav-5 fKAzIL">€3,000,000<!-- --> </span></div><p class="TitleBlock__Address-sc-1avkvav-8 dzihyY" data-testid="address">7 Henrietta Street, Dublin 1</p><div class="TitleBlock__CardInfo-sc-1avkvav-10 iCjViR" data-testid="card-info"><p class="TitleBlock__CardInfoItem-sc-1avkvav-9 iLMdur" data-testid="beds">6 Bed</p><p class="TitleBlock__CardInfoItem-sc-1avkvav-9 iLMdur" data-testid="baths">7 Bath</p><p class="TitleBlock__CardInfoItem-sc-1avkvav-9 iLMdur" data-testid="floor-area">763 m²</p><p class="TitleBlock__CardInfoItem-sc-1avkvav-9 cKZZql" data-testid="property-type">Terrace</p><div class="TitleBlock__BerContainer-sc-1avkvav-11 iXTpuT"><img alt="" class="TitleBlock__Ber-sc-1avkvav-7 iDkita" data-testid="ber-image" src="https://hermes.daft.ie/dsch-daft-frontend/1.0.1670857707/static/i

In [14]:
# all_data

In [15]:
# this cell: 
    # goes through every page on the website
    # gets the relevant information from each card and dds it to aa dictionary called listing
    # each listing is added to the list all_data 
    # it DOES NOT add the listings that are under the "ad" format --> need to fix this? this means we are missing approx 550 listings ... 
    
all_data = []

while True:
    
    try:
        html = await page.content()

        doc = BeautifulSoup(html)

        # content card returns a list with the block of information for each listing 
        content_card = doc.find_all(attrs={"class": ["Cardstyled__TitleBlockWrapper-nngi4q-4","eMeJos"]})

        # go through each of these to get the listing properties
        for card in content_card:
            try:
                listing = {}
                listing["price"] = card.find(attrs={"class": ["TitleBlock__StyledSpan-sc-1avkvav-5","fKAzIL"]}).string
                listing["address"] = card.find(attrs={"class": ["TitleBlock__Address-sc-1avkvav-8", "dzihyY"]}).string
                listing["bedrooms"] = card.find(attrs={"class": ["TitleBlock__CardInfoItem-sc-1avkvav-9", "iLMdur"], "data-testid": "beds"}).string
                listing["bathrooms"] = card.find(attrs={"class": ["TitleBlock__CardInfoItem-sc-1avkvav-9", "iLMdur"], "data-testid": "baths"}).string
                listing["floor_area"] = card.find(attrs={"class": ["TitleBlock__CardInfoItem-sc-1avkvav-9", "iLMdur"], "data-testid": "floor-area"}).text
                all_data.append(listing)
            except:
                pass

        print("Next page")
        async with page.expect_navigation():
            await page.get_by_text("Next >").click()
                           
    except Exception as e: 
        
        raise e
        print("Could not click the 'next' button")
        break
    
print("Done")

Next page
Next page
Next page
Next page
Next page
Next page
Next page
Next page
Next page
Next page
Next page
Next page
Next page


CancelledError: 

In [ ]:
# check check

# see how many listings I have 

print(len(all_data))

# await browser.close()

In [ ]:
import pandas as pd
from datetime import date

today = date.today()
year = today.year
month = today.month
day = today.day

if day < 9:
    day = "0" + str(day) 
    
df = pd.DataFrame.from_dict(all_data)

df.to_csv(f"./scrape_output/daft_listings_{year}{month}{day}.csv")

In [ ]:
# add missing price values for the first few cells manually 

In [ ]:
# await browser.close()

In [ ]:
df.tail()
